In [9]:
import torch
from snac import SNAC
from audiotools import AudioSignal
import torchaudio

model = SNAC.from_pretrained("hubertsiuzdak/snac_32khz").eval().cuda()
# audio = torch.randn(1, 1, 32000).cuda()  # placeholder for actual audio with shape (B, 1, T)
audio = AudioSignal('samples/output2.wav')
# audio, sample_rate = torchaudio.load("data/samples/output2.wav")
sample_rate = audio.sample_rate
print(sample_rate)
audio = audio.audio_data
# print(audio.shape, audio[0,0,-1000:])
audio = audio.cuda()

with torch.inference_mode():
    codes = model.encode(audio)
    audio_hat = model.decode(codes)

for code in codes:
    print(code.shape)
    print(code[0,1])
# print(audio_hat.shape)
torchaudio.save("output2.wav", audio_hat.squeeze(0).cpu(), sample_rate)

/home/patrick/anaconda3/envs/omini/lib/python3.10/site-packages/snac/snac.py:108: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path, map_locat

16000
torch.Size([1, 20])
tensor(983, device='cuda:0')
torch.Size([1, 40])
tensor(1540, device='cuda:0')
torch.Size([1, 80])
tensor(356, device='cuda:0')
torch.Size([1, 160])
tensor(933, device='cuda:0')


In [4]:
import torchaudio

waveform, sample_rate = torchaudio.load("data/samples/output2.wav")
print(waveform.shape)

torch.Size([1, 52566])


In [1]:
import torch
import dac
import torchaudio
from audiotools import AudioSignal

# Download a model
# model_path = dac.utils.download(model_type="44khz")
# model_path = dac.utils.download(model_type="24khz")
model_path = dac.utils.download(model_type="16khz")
model = dac.DAC.load(model_path)

model.to('cuda')

# Load audio signal file
signal = AudioSignal('data/samples/output2.wav', sample_rate=model.sample_rate)
# signal = AudioSignal("data/samples/output1_24.wav", sample_rate=model.sample_rate)
print(signal.shape)

# Encode audio signal as one long file
# (may run out of GPU memory on long files)
signal.to(model.device)
print(signal.sample_rate)
print(model.sample_rate)

with torch.no_grad():
    x = model.preprocess(signal.audio_data, signal.sample_rate)
    z, codes, latents, _, _ = model.encode(x)

    print(z.shape) # B D T
    print(codes.shape) # B N T
    print(latents.shape) # B N*D T

    # Decode audio signal
    y = model.decode(z)
    print(y.shape)
torchaudio.save("output2_dac.wav", y.squeeze(0).cpu(), signal.sample_rate)

# Alternatively, use the `compress` and `decompress` functions
# to compress long files.

signal = signal.cpu()
x = model.compress(signal)

print(x.codes.shape) # 1, 12, 340
# print(x.shape)

# # Save and load to and from disk
# x.save("compressed.dac")
# x = dac.DACFile.load("compressed.dac")

# Decompress it back to an AudioSignal
y = model.decompress(x)

# Write to file
y.write('output2-dac.wav')

/home/patrick/anaconda3/envs/omini/lib/python3.10/site-packages/audiotools/ml/layers/base.py:172: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_dict = torch.load(locat

torch.Size([1, 1, 52566])
16000
16000


NameError: name 'torch' is not defined

In [18]:
from audiotools import AudioSignal

a = AudioSignal("data/samples/output1_24.wav")

print(a.audio_data.dtype)
print(a.audio_data.min().item())
print(a.audio_data.max().item())
print(a.sample_rate)

torch.float32
-0.6456473469734192
0.6108736991882324
48000


In [2]:
from huggingface_hub import hf_hub_download
import torch
import torchaudio
from audiotools import AudioSignal

from moshi.models import loaders, LMGen
mimi_weight = hf_hub_download(loaders.DEFAULT_REPO, loaders.MIMI_NAME)
mimi = loaders.get_mimi(mimi_weight, device='cuda')
mimi.set_num_codebooks(8)  # up to 32 for mimi, but limited to 8 for moshi.

audio = AudioSignal('data/samples/output2.wav')
sample_rate, audio = audio.sample_rate, audio.audio_data # B, C, T -> 1, 1, T
audio = audio.cuda()

with torch.no_grad():
    codes = mimi.encode(audio)
    print(codes.shape)
    decoded = mimi.decode(codes)
    print(decoded.shape)

# torchaudio.save("output2_mimi.wav", decoded.squeeze(0).cpu(), sample_rate)
audio_signal = AudioSignal(decoded.cpu(), sample_rate=sample_rate)
audio_signal.write("output2_mimi_as.wav")

/home/patrick/anaconda3/envs/omini/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([1, 8, 28])
torch.Size([1, 1, 53760])


In [3]:
import whisper
import torch

def load_whisper(speech_path, input_type="mel", norm=True, mel_size=128):
    speech = whisper.load_audio(speech_path)
    if input_type == "raw":
        speech = torch.from_numpy(speech)
        if norm:
            speech = torch.nn.functional.layer_norm(speech, speech.shape)
    elif input_type == "mel":
        speech = whisper.pad_or_trim(speech)
        speech = whisper.log_mel_spectrogram(speech, n_mels=mel_size).permute(1, 0)
    
    return speech
        
speech = load_whisper("samples/output2.wav", input_type="raw")
print(speech.shape)

torch.Size([52566])
